# Problem

You own a paint factory. There are N different colors you can mix, and each color can be prepared "matte" or "glossy". So, you can make 2N different types of paint. Each of your customers has a set of paint types they like, and they will be satisfied if you have at least one of those types prepared. At most one of the types a customer likes will be a "matte". You want to make N batches of paint, so that:

* There is exactly one batch for each color of paint, and it is either matte or glossy.
* For each customer, you make at least one paint type that they like.
* The minimum possible number of batches are matte (since matte is more expensive to make).
* Find whether it is possible to satisfy all your customers given these constraints, and if it is, what paint types you should make.

If it is possible to satisfy all your customers, there will be only one answer which minimizes the number of matte batches.

# Input

One line containing an integer C, the number of test cases in the input file. For each test case, there will be:

* One line containing the integer N, the number of paint colors.
* One line containing the integer M, the number of customers.
* M lines, one for each customer, each containing:
* An integer T >= 1, the number of paint types the customer likes, followed by
     - T pairs of integers "X Y", one for each type the customer likes, where X is the paint color between 1 and N inclusive, and Y is either 0 to indicate glossy, or 1 to indicated matte. 

Note that:

* No pair will occur more than once for a single customer.
* Each customer will have at least one color that they like (T >= 1).
* Each customer will like at most one matte color. (At most one pair for each customer has Y = 1).

All of these numbers are separated by single spaces.

# Output

C lines, one for each test case in the order they occur in the input file, each containing the string 

* "Case #X: " where X is the number of the test case, starting from 1, followed by:
    - The string "IMPOSSIBLE", if the customers' preferences cannot be satisfied; OR
    - N space-separated integers, one for each color from 1 to N, which are 0 if the corresponding paint should be prepared glossy, and 1 if it should be matte.


# Limits

Small dataset:

* C = 100 
* 1 <= N <= 10  (number of paints)
* 1 <= M <= 100 (number of customers)

Large dataset:

* C = 5 
* 1 <= N <= 2000 
* 1 <= M <= 2000

The sum of all the T values for the customers in a test case will not exceed 3000.

# Sample

## Input:

In [418]:
input_lines = '''2 # number of test samples
5 # sample #1 number of colors
3 # sample #1 number of customers
1 1 1 # sample #1 customer #1 wants 1 type with color #1 to be matte 
2 1 0 2 0 # sample #1 customer #2 wants 2 types: first with color #1 to be glossy and color #2 to be glossy
1 5 0 # sample #1 customer #3 wants 1 type with color #5 to be glossy 
1 # sample #2 number of colors
2 # sample #2 number of customers
1 1 0 # sample #2 customer #1 wants 1 type with color #1 to be glossy
1 1 1 # sample #2 customer #2 wants 1 type with color #1 to be matte
'''

## Output:


    
    Case #1: 1 0 0 0 0   # make color #1 to be matte all others to be glossy 
    Case #2: IMPOSSIBLE 


In the first case, you must make color #1 matte, to satisfy the first customer. Every other paint type can be glossy. The second customer is satisfied by getting color #2 glossy, and the third customer is satisfied by getting color #5 glossy.
In the second case, there is only one color. One of your customers wants it matte and one wants it glossy. You cannot satisfy them both.

In [419]:
input_lines = '''5     #0
1                      #1 first
2                      #2
1 1 1                  #3
1 1 1                  #4 
1                      #5 second
2                      #6
1 1 0                  #7  
1 1 1                  #8 
5                      #9  third
3                      #10
1 1 1                  #11
2 1 0 2 0              #12
1 5 0                  #13
5                      #14 forth 
5                      #15
5 1 0 2 0 3 0 4 0 5 0  #16
1 3 1                  #17
2 2 0 5 1              #18 
1 4 1                  #19 
2 1 1 2 0              #20
5                      #21 fifth 
5                      #22
5 1 1 2 0 3 0 4 0 5 0  #23 
5 1 0 2 1 3 0 4 0 5 0  #24
5 1 0 2 0 3 1 4 0 5 0  #25
5 1 0 2 0 3 0 4 1 5 0  #26
5 1 0 2 0 3 0 4 0 5 1  #27
'''
if not isinstance(input_lines, list):
    input_lines = input_lines.split('\n')

Define helper which reads line and convert it to int if single value in line or a list of integers:

In [420]:
def process_line(line):
    line = line.split('#')[0].strip() # strip possible comment and empty litterals
    try:
        return int(line)  # if single value line
    except ValueError:
        return [int(x) for x in line.split()]

In [421]:
import pandas as pd
import numpy as np

class Customer:
    def __init__(self, line): # '3 1 0 2 0 5 1' -> ps.Series(data=[0, 0, 1], index=[1, 2, 5]) takes O(n_paints)
        array = process_line(line)
        data = []
        index = []
        for i in range(array[0]):
            index.append(array[2*i +1])
            data.append(array[2*i + 2])
        if sum(data) > 1:
            raise ValueError('customer wants more then 1 color in matte')
        self.series = pd.Series(data=data, index=index)
    
class Sample:
    def __init__(self, i, n_colors, customer_lines):
        self.sample_id = 'Case #{0}: '.format(i)
        self.is_impossible = False
        data = dict()
        for i, line in enumerate(customer_lines, start=1):
            customer = Customer(line)
            data['cus' + str(i)] = customer.series
        idx = range(1, n_colors + 1)
        self.data_frame = pd.DataFrame(data=data, index=idx)
        self.colors = pd.Series(data=[0] * n_colors, index=idx, name='colors') # initialy try all colors glossy 
        self.single_color = pd.DataFrame() # will collect here customers who want a single color
        self.drop_colors_with_no_customers(self.data_frame)
    
    @staticmethod
    def drop_colors_with_no_customers(data_frame):
        """Drops all colors which are not requested by any customers."""
        data_frame.dropna(axis='index', how='all', inplace=True)
        return data_frame
    
    def find_customers_with_single_color(self):
        """Find all columns with single NaN"""
        a = self.data_frame.count()
        a = a[a == 1]
        for col in a.index:
            self.single_color[col] = self.data_frame[col]
            self.data_frame.drop(col, axis='columns', inplace=True)
        self.drop_colors_with_no_customers(self.single_color)
        self.drop_colors_with_no_customers(self.data_frame) # drop no customers color again 
        return self.single_color    

    def how_to_sutisfy_single_color_customers(self):
        for index, row in self.single_color.iterrows():
            is_matte_in = 1 in row.values
            is_glossy_in = 0 in row.values
            if is_matte_in == is_glossy_in:
                print(self.sample_id + 'IMPOSSIBLE')
                self.is_impossible = True
                return
            else:
                matte_or_glossy = 1 if is_matte_in else 0
                self.colors[index] = matte_or_glossy
                self.colors.rename_axis({index: str(index) + 'fixed'}, inplace=True)

    def how_to_sutisfy_others(self):
        """At this point we have self.data_frame cleaned up 
           from colors which are not requested
           and customers with single color requriment
           self.colors has these single colors marke as xfixed 
        """
        if self.is_impossible:  # impossible combination found for customers with single color
            return
        
                    
    def execute(self):
        self.find_customers_with_single_color()
        self.how_to_sutisfy_single_color_customers()
        self.how_to_sutisfy_others()
        return self.colors

Now run actual code.

In [422]:
n_samples = process_line(input_lines[0])
samples = []
next_sample_shift = 1
for i in range(n_samples):
    n_colors = process_line(input_lines[next_sample_shift])
    n_customers = process_line(input_lines[next_sample_shift + 1])
    customer_lines = input_lines[ next_sample_shift + 2 : next_sample_shift + 2 + n_customers ]
    samples.append(Sample(i, n_colors, customer_lines))
    next_sample_shift += n_customers + 2

In [423]:
s1 = samples[0]
s2 = samples[1]
s3 = samples[2]
s4 = samples[3]
s5 = samples[4]

In [428]:
s5.data_frame

,cus1,cus2,cus3,cus4,cus5
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
5,0,0,0,0,1


In [431]:
s4.execute()

1         0
2         0
3fixed    1
4fixed    1
5         0
Name: colors, dtype: int64

In [432]:
s4.data_frame

,cus1,cus3,cus5
1,0,NaN,1
2,0,0,0
3,0,NaN,NaN
4,0,NaN,NaN
5,0,1,NaN


In [435]:
import ortools

ImportError: No module named 'ortools'